In [1]:
from keras.layers import *
from keras.models import Sequential,Model
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.optimizers import adam
from keras.layers.advanced_activations import LeakyReLU
import numpy as np

Using TensorFlow backend.


In [2]:
# Load Mnist Data
(X_train,_),(_,_)=mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [3]:
# Shape of our data 
X_train.shape

(60000, 28, 28)

In [0]:
# Reshape data from 60000*28*28 to 60000*28*28*1
X_train=X_train.reshape((*X_train.shape,1))

In [5]:
# Final Shape
X_train.shape

(60000, 28, 28, 1)

In [0]:
# Define batch size
BATCH_SIZE=256
NO_OF_BATCHES=int(60000/256)
# Define dimension of initial Fake Data 
NOISE_DIM=100
TOTAL_EPOCHS=50
HALF_BATCH=128
# Define loss
adam=adam(lr=2e-4,beta_1=0.5)

In [7]:
# Generator
generator=Sequential()

generator.add(Dense(7*7*128,input_shape=(NOISE_DIM,)))
generator.add(Reshape((7,7,128)))
generator.add(LeakyReLU(0.2))

generator.add(Conv2DTranspose(64,kernel_size=(5,5),strides=(2,2),padding="same"))
generator.add(LeakyReLU(0.2))

generator.add(Conv2DTranspose(1,kernel_size=(5,5),strides=(2,2),padding="same",activation="tanh"))
generator.compile(loss="binary_crossentropy",optimizer=adam)
generator.summary()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 64)        204864    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 28, 28, 1)         1601      
Total params: 839,937

In [8]:
# Discriminator
discriminator=Sequential()
discriminator.add(Conv2D(64,(5,5),strides=(2,2),padding='same',input_shape=(28,28,1)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Conv2D(64,(5,5),strides=(2,2),padding='same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Flatten())
discriminator.add(Dense(1,activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy',optimizer='adam')
discriminator.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          102464    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3137      
Total params: 107,265
Trainable params: 107,265
Non-trainable params: 0
________________________________________________

In [0]:
discriminator.trainable=False
gan_input=Input(shape=(NOISE_DIM,))
generated_image=generator(gan_input)
gan_output=discriminator(generated_image)

#Functional API
model=Model(gan_input,gan_output)
model.compile(loss="binary_crossentropy",optimizer=adam)



In [0]:
# function to save final generated images
!mkdir images model
def save_imgs(epoch,samples=100):

  noise=np.random.normal(0,1,size=(samples,NOISE_DIM))
  generated_img=generator.predict(noise)
  generated_imgs=generated_img.reshape(samples,28,28)

  plt.figure(figsize=(10,10))
  for i in range(samples):
    plt.subplot(10,10,i+1)
    plt.imshow(generated_imgs[i],interpolation="nearest",cmap="gray")
    plt.axis("off")
  
  plt.tight_layout()
  plt.savefig("images/gan_output_epoch_{0}.png".format(epoch+1))
  plt.show()

In [0]:
#Training Loop
for epoch in range(50):
  epoch_d_loss=0
  epoch_g_loss=0

  #Mini Batch SGD
  for step in range(NO_OF_BATCHES):
    #Step1-Train Discriminator
    #50% Real Data +50% of fake data
    #idx=np.random.randint(low,high,no_of_numbers)
    idx=np.random.randint(0,X_train.shape[0],HALF_BATCH)
    real_imgs=X_train[idx]


    #FAKE data
    noise=np.random.normal(0,1,size=(HALF_BATCH,NOISE_DIM))
    fake_imgs=generator.predict(noise) #Forward

    #Lables
    real_y=np.ones((HALF_BATCH,1))*0.9 # One sided label smoothing for discriminator
    fake_y=np.zeros((HALF_BATCH,1))

    #Train our Discriminator
    d_loss_real=discriminator.train_on_batch(real_imgs,real_y)
    d_loss_fake=discriminator.train_on_batch(fake_imgs,fake_y)
    d_loss=0.5*d_loss_real + 0.5*d_loss_fake

    epoch_d_loss+=d_loss

    #Train Generator (Considering Frozen Discriminator)
    noise=np.random.normal(0,1,size=(BATCH_SIZE,NOISE_DIM))
    ground_truth_y=np.ones((BATCH_SIZE,1))
    g_loss=model.train_on_batch(noise,ground_truth_y)
    epoch_g_loss+=g_loss

  print("Epoch %d Disc Loss %.4f generator Loss %.4f"%((epoch+1),epoch_d_loss/NO_OF_BATCHES,epoch_g_loss/NO_OF_BATCHES))
  save_imgs(epoch)
  if((epoch+1)%5==0):
    generator.save("model/gan_generator_{0}.h5".format(epoch+1))

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
